In [1]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = './../data' 
STUDY_DIR = DATA_DIR + '/study'


#users = pd.read_csv(f'{DATA_DIR}/users.csv.gz')

# use study for less data, for testing
events = pd.read_csv(f'{DATA_DIR}/events.csv.gz')
#transactions = pd.read_csv(f'{DATA_DIR}/transactions.csv.gz')

: 

: 

In [3]:
from itemknn_model import ItemKNN
from data import ItemKNNSplitter

splitter = ItemKNNSplitter(events)


model = ItemKNN(5)
model.train(splitter.get_matrix())

In [4]:
for sample in splitter.get_test_samples():
    model.test_step(sample)

In [5]:
model.predict_proba

tensor([ 1.5188e+01,  4.4845e+01, -9.5294e+00, -4.1922e+01,  3.0618e+01,
        -2.2926e+01,  0.0000e+00,  0.0000e+00,  6.0454e+00, -3.6847e+01,
         4.8257e+01,  0.0000e+00,  5.4167e-01,  2.1227e+01, -1.2461e+01,
         1.3955e+01,         nan,  0.0000e+00, -6.7115e+00,  2.2637e+01,
         0.0000e+00,  0.0000e+00, -2.8357e+01, -3.6490e+01,  0.0000e+00,
         1.3991e+01,  2.6231e+01, -8.7333e+01,  0.0000e+00, -4.7397e+01,
         1.8471e+01,  1.9567e+02,  0.0000e+00, -2.2926e+01,  1.8618e+02,
        -1.7963e+01, -3.0377e+02,  1.7519e+02,  0.0000e+00, -3.0709e+01,
         3.0722e+01,  1.3155e+02, -1.3396e+01, -3.8590e+01,  0.0000e+00,
         9.4435e+00, -1.6532e+01,  3.1183e-01,  0.0000e+00,         nan,
         0.0000e+00,  0.0000e+00, -2.4793e+01, -1.1022e+01,  0.0000e+00,
        -2.3514e+01, -1.5504e+01,  1.3366e+01, -1.9124e+00, -1.4757e+01,
        -4.3078e+01,  3.4401e+01,  0.0000e+00,  2.7196e+01,  0.0000e+00,
        -3.9709e+01, -1.1693e+01, -1.2126e+03,  4.3

In [6]:
from predictions import get_predictions, write_outputs

# TODO: change path for saving file
write_outputs(get_predictions(splitter.get_test_samples(), model.predict_proba), [], 'itemknn', '.')